In [1]:
#import matplotlib
import numpy
import numpy.random as npr
from random import *
#from matplotlib import *
import csv
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

i = datetime.now()
version = i.strftime('%Y%m%d %H%M%S')

In [2]:
#def save_result(matrice, parametres):
#    filename = parametres
#    print (filename)
#    with open(filename, 'wb') as csvfile:
#        tableau = matrice
#        filename = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#        filename.writerows(tableau)

def deriv_2(f, x, y, dx):
    return (f(x+dx, y)-f(x, y))/dx

def deriv_3(f, x, y, z, dx):
    return (f(x+dx, y, z)-f(x, y, z))/dx

def F(a, b):                             # point fixe avec et sans lansing b < d
    u = 0
    if a == 0:
        u = b - 1
    else:
        u = ((1/a)*(1-np.exp(-a*b)))-1
    return u

def H(a,b,d):                           # point fixe sans lansing b > d
    u = 0
    if a == 0:
        u = d - np.exp(d-b)
    else:
        u = ((1-np.exp(-a*d))/a) + (np.exp(d)/(1+a))*(np.exp(-d*(1+a))-np.exp(-b*(1+a)))-1
    return u

def G(a,d):                           # point fixe sans lansing b > d
    u = 0
    if a == 0:
        u = d - 1
    else:
        u = ((1/a)*(1-np.exp(-a*d)))-1
    return u



##methode de newton sans effet lansing
##renvoie la fitness associee au trait (b,d). x_{0}, epsilon, delta sont les parametres pour la methode de newton
#x_0 initialisations (prendre 0.5 par exemple) et epsilon, delta a choisir petits
def malthus_nl(b, d, x_0, epsilon, delta):
    if b <= d:
        u = x_0
        while abs(F(u, b)) > epsilon:
            u += - (F(u, b)/deriv_2(F, u, b, delta))
        return u
    else:
        v = x_0
        while abs(H(v, b, d)) > epsilon :
            v += - (H(v, b, d)/deriv_3(H, v, b, d, delta))
        return v



#avec effet lansing
def malthus_l(b, d, x_0, epsilon, delta):
    if b <= d:
        u = x_0
        while abs(F(u, b)) > epsilon:
            u += - (F(u, b) / deriv_2(F, u, b, delta))
        return u
    else:
        v = x_0
        while abs(G(v, d)) > epsilon:
            v += - (G(v, d) / deriv_2(G, v, d, delta))
        return v





def save_result(matrice_pop):
    filename = 'malthus.csv'
    with open(filename, 'w') as csvfile:
        colnames = ['time', 'lansing', 'no lansing']
        tableau = numpy.vstack((colnames, matrice_pop))
        filename = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filename.writerows(tableau)

In [3]:
#dynamique parametre malthusien moyen sur un ensemble de simu
def dyn_malthus_glob(nombre_simu, nombre_instant, fichier):

    H = [0] * nombre_simu
    for y in range(nombre_simu):
        r = numpy.genfromtxt(('%i' + fichier + '.csv') % (y), dtype=float, delimiter=';',
                             names=True, )  # tableau excel -> tableau numpy
        R = numpy.zeros((len(r), 7))  # tableau numpy de la population
        for t in range(len(r)):
            for j in range(7):
                R[t, j] = r[t][j]
        H[y] = R

    a = numpy.sort(numpy.random.choice(R[1000:, 2], nombre_instant), axis=None)
    malth = numpy.zeros((len(a), 3))
    malth[:, 0] = a

    for i in range(len(a)):

        x = numpy.logical_and(H[0][:, 2] <= a[i], a[i] < H[0][:, 3])
        xl = numpy.logical_and(x, H[0][:, 6] == 1)
        xnl = numpy.logical_and(x, H[0][:, 6] == 0)

        malth_lans = numpy.zeros(len(H[0][xl, 0]))
        for j in range(len(malth_lans)):
            malth_lans[j] = malthus_l(H[0][xl, 0][j], H[0][xl, 1][j], 0.5, 0.00001, 0.000001)
        malth_lans = malth_lans[~numpy.isnan(malth_lans)]
        malth_lans = malth_lans[~numpy.isinf(malth_lans)]
        malth_lans = malth_lans[(malth_lans > 0)]

        malth[i, 1] = numpy.mean(malth_lans)

        malth_nolans = numpy.zeros(len(H[0][xnl, 0]))

        for j in range(len(malth_nolans)):

            malth_nolans[j] = malthus_nl(H[0][xnl, 0][j], H[0][xnl, 1][j], 0.5, 0.00001, 0.000001)

        malth[i, 2] = numpy.mean(malth_nolans)

        for y in range(nombre_simu):
            print(y)
            x = numpy.logical_and(H[y][:, 2] <= a[i], a[i] < H[y][:, 3])
            xl = numpy.logical_and(x, H[y][:, 6] == 1)
            xnl = numpy.logical_and(x, H[y][:, 6] == 0)

            malth_lans = numpy.zeros(len(H[y][xl, 0]))
            for j in range(len(malth_lans)):
                malth_lans[j] = malthus_l(H[y][xl, 0][j], H[y][xl, 1][j], 0.5, 0.00001, 0.000001)
            malth_lans = malth_lans[~numpy.isnan(malth_lans)]
            malth_lans = malth_lans[~numpy.isinf(malth_lans)]
            malth_lans = malth_lans[(malth_lans > 0)]
            malth[i, 1] += numpy.mean(malth_lans)
            malth_nolans = numpy.zeros(len(H[y][xnl, 0]))
            for j in range(len(malth_nolans)):
                malth_nolans[j] = malthus_nl(H[y][xnl, 0][j], H[y][xnl, 1][j], 0.5, 0.00001, 0.000001)
            malth[i, 2] += numpy.mean(malth_nolans)


        malth[i, 1] = malth[i, 1]/nombre_simu
        malth[i, 2] = malth[i, 2]/nombre_simu

    return(malth)
    #print(malth)
    #fig = plt.figure(figsize=(18, 18))
    #fig.suptitle('', fontsize=35)
    #fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.85, wspace=0.5, hspace=0.5)
    #plt.subplot(111)
    #plt.scatter(malth[:, 0], malth[:, 1], s=5, c='red', label='Population with Lansing effect', edgecolor='', alpha=1.0)
    #plt.scatter(malth[:, 0], malth[:, 2], s=5, c='blue', label='Population without Lansing effect', edgecolor='',
    #            alpha=1.0)
    #plt.axis(
    #    [0, max(malth[:, 0]) + 1, min(min(malth[:, 1]), min(malth[:, 2])), max(max(malth[:, 1]), max(malth[:, 2]))])
    #plt.xlabel('time', fontsize=30)
    #plt.ylabel('fitness', fontsize=30)

In [ ]:
A = dyn_malthus_glob(2, 5 , 'bmbdmvieil,1.51.31.50.82,c9e-05,p0.1,v0.05,ns200000,tp1000')
print(A)
save_result(A)